# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [195]:
# Libraries

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [196]:
# your code here
import pandas as pd
import numpy as np

df = pd.read_csv("data\Admission_Predict.csv")
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [197]:
# your code here
df.columns = df.columns.str.strip()
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

In [198]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Let's evaluate the dataset by looking at the `head` function.

In [199]:
# your code here
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [200]:
# your code here
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [201]:
df.set_index('Serial No.',inplace=True)

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [202]:
df.dtypes
df1 = df.astype({"GRE Score":"string"})
df2 = df1.astype({"CGPA":"string"})
df2.dtypes

df2["GRE +  Score"] = df2["GRE Score"] + df2["CGPA"]
df2["GRE +  Score"].value_counts()

# after converting both columns to string and summing them, the valuecounts() function tells me that every single value is repeated just one time, which make the column unique

3379.65    1
3168.0     1
3228.67    1
3148.21    1
3309.34    1
          ..
3249.04    1
3259.11    1
3309.45    1
3128.78    1
3339.66    1
Name: GRE +  Score, Length: 385, dtype: Int64

In [209]:
df2

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE + Score
Serial No.,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,3379.65
2,316,104,3,3.0,3.5,8.0,1,0.72,3168.0
3,322,110,3,3.5,2.5,8.67,1,0.80,3228.67
4,314,103,2,2.0,3.0,8.21,0,0.65,3148.21
5,330,115,5,4.5,3.0,9.34,1,0.90,3309.34
...,...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82,3249.04
382,325,107,3,3.0,3.5,9.11,1,0.84,3259.11
383,330,116,4,5.0,4.5,9.45,1,0.91,3309.45


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [228]:
df2.reset_index(drop=False, inplace=True)
df2

,GRE + Score,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,3379.65,1,337,118,4,4.5,4.5,9.65,1,0.92
1,3168.0,2,316,104,3,3.0,3.5,8.0,1,0.72
2,3228.67,3,322,110,3,3.5,2.5,8.67,1,0.80
3,3148.21,4,314,103,2,2.0,3.0,8.21,0,0.65
4,3309.34,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...,...
380,3249.04,381,324,110,3,3.5,3.5,9.04,1,0.82
381,3259.11,382,325,107,3,3.0,3.5,9.11,1,0.84
382,3309.45,383,330,116,4,5.0,4.5,9.45,1,0.91
383,3128.78,384,312,103,3,3.5,4.0,8.78,0,0.67


In [230]:
df2.set_index("GRE +  Score", inplace=True)


In [245]:
df2#.drop(['GRE Score'])
df2.columns
df2.drop(columns=["GRE Score", "CGPA"],inplace=True)

In [247]:
df2.reset_index(drop=False, inplace=True)
df2

,GRE + Score,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,3379.65,1,118,4,4.5,4.5,1,0.92
1,3168.0,2,104,3,3.0,3.5,1,0.72
2,3228.67,3,110,3,3.5,2.5,1,0.80
3,3148.21,4,103,2,2.0,3.0,0,0.65
4,3309.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...
380,3249.04,381,110,3,3.5,3.5,1,0.82
381,3259.11,382,107,3,3.0,3.5,1,0.84
382,3309.45,383,116,4,5.0,4.5,1,0.91
383,3128.78,384,103,3,3.5,4.0,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [250]:
df.reset_index(drop=False, inplace=True)

In [290]:
# your code here

cond1 = df["CGPA"] > 9
cond2 = df["Research"] == 1

df_cond = df[cond1 & cond2]
df_cond.count()

# there are 101 rows where the CGPA is higher than 9 and the student has conducted a research

Serial No.           101
GRE Score            101
TOEFL Score          101
University Rating    101
SOP                  101
LOR                  101
CGPA                 101
Research             101
Chance of Admit      101
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [292]:
# your code here
cond1 = df["CGPA"] > 9
cond2 = df["SOP"] < 3.5
df_cond = df[cond1 & cond2]
df_cond

df["Chance of Admit"].mean()

# the chance of admit for these aplicants is aprox. 72%

0.7239220779220773

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [320]:
df.head(2)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72


In [314]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """

    # your code here

    col = (col - np.mean(col))/ np.std(col)
    return col


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [321]:
# your code here
df["CGPA_std"] = standardize(df["CGPA"])
df["GRE_std"] = standardize(df["GRE Score"])
df["LOR_std"] = standardize(df["LOR"])
df.head(2)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840


We will generate the decision choice at random using the code below. Please run the cell.

In [207]:
# Libraries
from random import choices

In [322]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [331]:
# your code here

df["deciding_column"] = df.lookup(df.index, decision_choice)
df.head()

C:\Users\joaol\AppData\Local\Temp\ipykernel_4664\3620949174.py:3: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  df["deciding_column"] = df.lookup(df.index, decision_choice)


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.456297
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [332]:
# your code here
def admission(x):
    if x > 0.8:
        return "1"
    else:
        return "0"

df["decision"] = df["deciding_column"].apply(admission)
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.456297,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [337]:
# your code here
condition = df["decision"] == "1"
df[condition].count()

# 89 applicants were accepted, out of 385.

Serial No.           89
GRE Score            89
TOEFL Score          89
University Rating    89
SOP                  89
LOR                  89
CGPA                 89
Research             89
Chance of Admit      89
CGPA_std             89
GRE_std              89
LOR_std              89
deciding_column      89
decision             89
dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [393]:
# your code here
df.columns = [i.lower().replace(" ", "_") for i in df.columns]
df.columns
# i dont know how to remove all non character symbols with underscores

Index(['serial_no.', 'gre_score', 'toefl_score', 'university_rating', 'sop',
       'lor', 'cgpa', 'research', 'chance_of_admit', 'cgpa_std', 'gre_std',
       'lor_std', 'deciding_column', 'decision'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [361]:
# your code here
condition_a = df["University Rating"] >= 4
df_a = df[condition_a]
df_a["GRE Score"] = df_a["GRE Score"] + 10
df_a["GRE Score"]

C:\Users\joaol\AppData\Local\Temp\ipykernel_4664\1010821773.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a["GRE Score"] = df_a["GRE Score"] + 10


0      347
4      340
9      337
10     338
18     335
      ... 
370    345
377    336
379    339
382    340
384    343
Name: GRE Score, Length: 126, dtype: int64

In [366]:
adjusted_gre = pd.cut(df_a['GRE Score'],4).value_counts()
adjusted_gre

(337.5, 350.0]     65
(325.0, 337.5]     45
(312.5, 325.0]     14
(299.95, 312.5]     2
Name: GRE Score, dtype: int64